In [20]:
import os
import numpy as np
import matplotlib.pyplot as plt


from scipy.optimize import broyden1

from edpyt.shared import params
from edpyt.espace import build_espace, screen_espace
from edpyt.gf2_lanczos import build_gf2_lanczos
new_path = '/capstor/scratch/cscs/ajayaraj/qtpyt-tests/pentadienyl'
os.chdir(new_path)

In [21]:
# === Load inputs ===
input_folder = "output/lowdin"
output_folder = "output/lowdin/beta_1000/ed"
os.makedirs(output_folder, exist_ok=True)

H_eff = np.load(f"{input_folder}/effective_hamiltonian.npy")
occupancy_goal = np.load(f"{input_folder}/beta_1000/occupancies.npy")
V = np.loadtxt(f"{input_folder}/U_matrix.txt")

# === Parameters ===
nimp = H_eff.shape[0]
eta = 1e-3
beta = 1000

# === Initial double counting ===
DC0 = np.diag(V.diagonal() * (occupancy_goal - 0.5))
neig = np.ones((nimp + 1) * (nimp + 1), int) * 6

params["z"] = occupancy_goal

de = 0.01
energies = np.arange(-0.5, 1 + de / 2.0, de).round(7)
eta = 1e-3

dc_diag_optimized = np.load(f"{output_folder}/ed_dcc_diag.npy")
DC = np.diag(dc_diag_optimized)

In [ ]:
class Sigma:
    def __init__(self, gf0, gf, H_eff, eta=1e-5):
        self.gf0 = gf0
        self.gf = gf
        self.eta = eta
        self.H_eff = H_eff

    def retarded(self, energy):
        energies = np.atleast_1d(energy)
        g0 = self.gf0(energies, self.eta)
        g = self.gf(energies, self.eta)
        print("g0", g0.shape)
        print("g", g.shape)
        sigma = np.empty((energies.size, self.gf.n, self.gf.n), complex)
        for e, energy in enumerate(energies):
            print("e", e)
            print("energy", energy)
            print("g0", g0[..., e].shape)
            print("g", g[..., e].shape)
            sigma[e] = np.linalg.inv(g0[..., e]) - np.linalg.inv(g[..., e])
            # sigma[e] = energy - self.H_eff - np.linalg.inv(g[..., e])
        return sigma

In [25]:
espace0, egs0 = build_espace(H_eff, np.zeros_like(H_eff), neig_sector=neig)
screen_espace(espace0, egs0, beta)
gf0 = build_gf2_lanczos(H_eff, np.zeros_like(H_eff), espace0, beta, egs0)

espace, egs = build_espace(H_eff - DC, V, neig_sector=neig)
screen_espace(espace, egs, beta)
gf = build_gf2_lanczos(H_eff - DC, V, espace, beta, egs)

sigma = Sigma(gf0, gf, H_eff, eta=3e-2)
test_energy = np.array([-3.0])
sigma_ret = sigma.retarded(test_energy)
print("Sigma", np.round(sigma_ret,5))

g0 (7, 7, 1)
g (7, 7, 1)
e 0
energy -3.0
g0 (7, 7)
g (7, 7)
Sigma [[[ 0.10157-0.03168j -2.70505-0.00377j -2.83743-0.00129j
   -3.01147-0.00032j -3.05411+0.00124j -2.92171-0.0038j
   -2.96356-0.00255j]
  [-2.70505-0.00377j -0.04797-0.04093j -2.02328-0.0035j
   -3.15505-0.00135j -3.19337+0.00343j -2.71244-0.01007j
   -2.86582-0.00687j]
  [-2.83743-0.00129j -2.02328-0.0035j  -0.09649-0.03204j
   -3.75493+0.00013j -2.98915+0.00131j -2.78222-0.00423j
   -2.89929-0.00281j]
  [-3.01147-0.00032j -3.15505-0.00135j -3.75493+0.00013j
   -0.20596-0.03424j -3.91158+0.00046j -2.81285+0.00151j
   -2.97871+0.00053j]
  [-3.05411+0.00124j -3.19337+0.00343j -2.98915+0.00131j
   -3.91158+0.00046j -0.08599-0.03209j -4.03441+0.00366j
   -3.25582+0.0025j ]
  [-2.92171-0.0038j  -2.71244-0.01007j -2.78222-0.00423j
   -2.81285+0.00151j -4.03441+0.00366j -0.07817-0.04578j
   -2.43908-0.00985j]
  [-2.96356-0.00255j -2.86582-0.00687j -2.89929-0.00281j
   -2.97871+0.00053j -3.25582+0.0025j  -2.43908-0.00985j
    0.

In [15]:
sigma_ref = np.load("reference/sigma_ed.npy")
np.round(sigma_ref[0],5)

array([[ 0.042  -5.400e-04j, -0.20999+1.410e-03j, -0.14253+7.000e-05j,
        -0.05437-1.750e-03j,  0.03506-2.000e-04j,  0.06826+1.400e-03j,
         0.02516+6.600e-04j],
       [-0.20999+1.410e-03j, -0.14289-5.570e-03j,  0.93223-2.300e-04j,
         0.32793+6.620e-03j, -0.13489+7.500e-04j, -0.29843-5.310e-03j,
        -0.1158 -2.520e-03j],
       [-0.14253+7.000e-05j,  0.93223-2.300e-04j,  0.05644-6.200e-04j,
         0.75422+1.000e-04j,  0.02843+1.600e-04j, -0.16886+4.000e-05j,
        -0.06718+3.000e-05j],
       [-0.05437-1.750e-03j,  0.32793+6.620e-03j,  0.75422+1.000e-04j,
        -0.31032-1.104e-02j,  0.87969-9.600e-04j,  0.40881+8.540e-03j,
         0.11936+4.020e-03j],
       [ 0.03506-2.000e-04j, -0.13489+7.500e-04j,  0.02843+1.600e-04j,
         0.87969-9.600e-04j,  0.06827-8.300e-04j,  1.01447+6.600e-04j,
         0.23074+3.000e-04j],
       [ 0.06826+1.400e-03j, -0.29843-5.310e-03j, -0.16886+4.000e-05j,
         0.40881+8.540e-03j,  1.01447+6.600e-04j, -0.26852-8.830e-03j